In [ ]:
import xml.etree.ElementTree as ET
import networkx as nx
import os
from math import sqrt
import matplotlib.pyplot as plt
from collections import deque, Counter

# Function to parse XPDL and order transitions correctly
def parse_xpdl(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    ns = {'xpdl': 'http://www.wfmc.org/2009/XPDL2.2'}
    
    # Lists
    pools = []
    lanes = []
    activities = []
    transitions = []  # Now using a list with manual duplicate check
    associations = []
    message_flows = []
    activity_id_to_name = {}  # Mapping from activity ID to name
    id_to_activity = {}  # Store activity details for reference
    
    # Parse Pools
    for pool in root.findall('.//xpdl:Pool', ns):
        pool_name = pool.get('Name')
        if pool_name:
            pools.append({'name': pool_name})
    
    # Parse Lanes
    for lane in root.findall('.//xpdl:Lane', ns):
        lane_name = lane.get('Name')
        if lane_name:
            lanes.append({'name': lane_name})
    
    # Parse Activities
    for activity in root.findall('.//xpdl:Activity', ns):
        activity_id = activity.get('Id')
        activity_name = activity.get('Name')
        event = activity.find('.//xpdl:Event', ns)
        route = activity.find('.//xpdl:Route', ns)
        gateway_type = route.get('GatewayType') if route is not None else 'None'
        is_start_event = False
        is_end_event = False
        
        if event is not None:
            if event.find('xpdl:StartEvent', ns) is not None:
                activity_name = 'StartEvent'
                is_start_event = True
            elif event.find('xpdl:EndEvent', ns) is not None:
                activity_name = 'EndEvent'
                is_end_event = True

        if activity_id:
            activity_detail = {
                'id': activity_id,
                'name': activity_name,
                'type': gateway_type,
                'is_start_event': is_start_event,
                'is_end_event': is_end_event,
                'from': [],
                'to': [],
                'data_objects': []  # Initialize an empty list for attached data objects
            }
            id_to_activity[activity_id] = activity_detail
            activity_id_to_name[activity_id] = activity_name if activity_name else activity_id

    # Parse DataObjects and assign them to the corresponding Activity
    for data_object in root.findall('.//xpdl:DataObject', ns):
        data_id = data_object.get('Id')
        data_name = data_object.get('Name')
        if data_id and data_name:
            for activity in id_to_activity.values():
                activity['data_objects'].append({'id': data_id, 'name': data_name, 'type': 'DataObject'})

    # Parse DataStores and assign them to the corresponding Activity
    for data_store in root.findall('.//xpdl:DataStore', ns):
        data_id = data_store.get('Id')
        data_name = data_store.get('Name')
        if data_id and data_name:
            for activity in id_to_activity.values():
                activity['data_objects'].append({'id': data_id, 'name': data_name, 'type': 'DataStore'})

            
    # Parse Transitions and create a transition map
    transition_map = {}  # Map each source to its target(s)
    reverse_transition_map = {}  # Map each target to its source(s)
    for transition in root.findall('.//xpdl:Transition', ns):
        source = transition.get('From')
        target = transition.get('To')
        if source and target:
            if source not in transition_map:
                transition_map[source] = []
            transition_map[source].append(target)

            if target not in reverse_transition_map:
                reverse_transition_map[target] = []
            reverse_transition_map[target].append(source)

            # Update `from` and `to` references
            if source in id_to_activity:
                id_to_activity[source]['to'].append(target)
            if target in id_to_activity:
                id_to_activity[target]['from'].append(source)
           
    # Add Associations to Transitions
    for association in root.findall('.//xpdl:Association', ns):
        source = association.get('Source')
        target = association.get('Target')
        name = "data"
        if source in id_to_activity:
            for data_object in id_to_activity[source]["data_objects"]:
                if data_object["id"] == target:
                    name = data_object["name"]
        else:
            print(f"Warning: Association source {source} not found in id_to_activity.")
        if source and target:
            associations.append({'from': activity_id_to_name.get(source, source), 'to': name})

    # Add Message Flows to Transitions
    for message_flow in root.findall('.//xpdl:MessageFlow', ns):
        source = message_flow.get('Source')
        target = message_flow.get('Target')
        name = "data"
        if source in id_to_activity:
            for data_object in id_to_activity[source]["data_objects"]:
                if data_object["id"] == target:
                    name = data_object["name"]
        else:
            print(f"Warning: MessageFlow source {source} not found in id_to_activity.")
        if source and target:
            message_flows.append({'from': activity_id_to_name.get(source, source), 'to': name})

                
    # Find the StartEvent node
    start_node = next((act for act in id_to_activity.values() if act['is_start_event']), None)
    if not start_node:
        raise ValueError("No StartEvent found in the XPDL.")

    # Track visited nodes
    visited = set()
    ordered_transitions = []  # Store transitions in order

    # Traverse the transitions in correct order using BFS
    queue = deque([start_node['id']])
    while queue:
        current_id = queue.popleft()
        if current_id in visited:
            continue
        visited.add(current_id)

        if current_id in transition_map:
            for target_id in transition_map[current_id]:
                transition_entry = {
                    'from': activity_id_to_name.get(current_id, current_id),
                    'to': activity_id_to_name.get(target_id, target_id)
                }

                # **Manually check for duplicates before adding**
                if transition_entry not in ordered_transitions:
                    ordered_transitions.append(transition_entry)
                queue.append(target_id)

    # Detect missing transitions
    missing_transitions = []
    for activity in id_to_activity.keys():
        if activity not in visited:
            missing_transitions.append(activity)

    # **Step 1: Reconstruct missing transitions using XPDL `from` and `to`**
    if missing_transitions:
        print("\n⚠️ Missing transitions detected! Attempting to reconstruct...")
        for missing_id in missing_transitions:
            missing_activity = id_to_activity[missing_id]
            print(f"  - {missing_activity['name']} ({missing_id}) is missing connections.")

            # Find the correct `from` node from the original XPDL transition list
            if missing_id in reverse_transition_map:
                correct_from = reverse_transition_map[missing_id][0]  # Take the first known connection
                correct_from_name = activity_id_to_name.get(correct_from, correct_from)

                transition_entry = {
                    'from': correct_from_name,
                    'to': activity_id_to_name.get(missing_id, missing_id)
                }
                if transition_entry not in ordered_transitions:  # Prevent duplicates
                    ordered_transitions.append(transition_entry)
                    print(f"  ✅ Reconstructed missing transition: {correct_from_name} → {missing_activity['name']}")

            if missing_id in transition_map:
                correct_to = transition_map[missing_id][0]  # Take the first known connection
                correct_to_name = activity_id_to_name.get(correct_to, correct_to)

                transition_entry = {
                    'from': activity_id_to_name.get(missing_id, missing_id),
                    'to': correct_to_name
                }
                if transition_entry not in ordered_transitions:  # Prevent duplicates
                    ordered_transitions.append(transition_entry)
                    print(f"  ✅ Reconstructed missing transition: {missing_activity['name']} → {correct_to_name}")

    # **Step 3: Append Associations & Message Flows in Order**
    ordered_transitions.extend(associations)
    ordered_transitions.extend(message_flows)
                    
    print("Final Transitions:", file_path, ordered_transitions)

    return {
        'pools': pools,
        'lanes': lanes,
        'activities': list(id_to_activity.values()),
        'transitions': ordered_transitions,  # Use ordered transitions
        'associations': associations,
        'message_flows': message_flows
    }


def convert_to_interstructural_with_order(parsed_data):
    """
    Convert parsed XPDL data into a graph preserving the original process flow.
    Activities and transitions are added in the correct order.
    """
    G = nx.DiGraph()

    # Map activity IDs to their details for quick access
    activity_map = {activity['id']: activity for activity in parsed_data['activities']}

    # Step 1: Add all activities first but connect them through transitions
    print("parsed_data['transitions']", parsed_data['transitions'])
    for transition in parsed_data['transitions']:
        source_id = transition['from']
        target_id = transition['to']

        # Ensure the source and target activities exist
        source_activity = activity_map.get(source_id, {'name': source_id, 'type': 'Unknown'})
        target_activity = activity_map.get(target_id, {'name': target_id, 'type': 'Unknown'})
        print("source_id->", source_id, "|||", source_activity)
        print("target_id->", target_id, "|||", target_activity)

        if not source_activity['name']:
            source_activity['name'] = str(source_activity["type"]) + "_" + source_activity["id"]
            
        if not target_activity['name']:
            target_activity['name'] = str(target_activity["type"]) + "_" + target_activity["id"]
        
        # Add source activity if not already added
        if not G.has_node(source_activity['name']):
            G.add_node(source_activity['name'], label=source_activity['name'], type=source_activity.get('type', 'Activity'))
            
        # Add transition node
        transition_node = f"Transition_{source_activity['name']}_to_{target_activity['name']}"
        G.add_node(transition_node, label=transition_node, type="Transition")    
        
        # Add target activity if not already added
        if not G.has_node(target_activity['name']):
            G.add_node(target_activity['name'], label=target_activity['name'], type=target_activity.get('type', 'Activity'))

        # Connect source -> transition -> target
        G.add_edge(source_activity['name'], transition_node)  # Source to Transition
        G.add_edge(transition_node, target_activity['name'])  # Transition to Target

    return G

def text_to_vector(text):
    """
    Converts a string into a frequency vector.
    """
    words = text.lower().split()
    return Counter(words)

def cosine_similarity(text1, text2):
    """
    Calculates cosine similarity between two text strings.
    """
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    # print(vector1, vector2)

    intersection = set(vector1.keys()) & set(vector2.keys())
    dot_product = sum([vector1[word] * vector2[word] for word in intersection])

    magnitude1 = sqrt(sum([count ** 2 for count in vector1.values()]))
    magnitude2 = sqrt(sum([count ** 2 for count in vector2.values()]))

    if not magnitude1 or not magnitude2:
        return 0.0

    return dot_product / (magnitude1 * magnitude2)

def calculate_semantic_similarity(target_node, other_node, component):
    """
    Calculates semantic similarity between two nodes using cosine similarity.
    Missing sub-components are skipped, and the result is normalized by the total weight.
    """

    # Adjusted weights with 'label' and normalized to sum to 1
    sub_components_weights = {
        'pools': (['label'], [1.0]),
        'lanes': (['label'], [1.0]),
        'activities': (['label', 'type'], [0.5, 0.5]),
        'data_objects': (['label', 'type'], [0.5, 0.5]),
        'transitions': (['type'], [1.0]),  # Only check 'type' for transitions
        'associations': (['label'], [1.0]),
        'message_flows': (['label'], [1.0])
    }

    if component not in sub_components_weights:
        return 0  # If the component type doesn't match, return 0 similarity

    sub_components, weights = sub_components_weights[component]
    similarities = []
    existing_weights = 0  # To normalize only the used weights

    # Calculate weighted cosine similarity for each sub-component
    for sub_component, weight in zip(sub_components, weights):
        target_value = target_node.get(sub_component, None)
        other_value = other_node.get(sub_component, None)

        # Skip if either value is missing
        if target_value is None or other_value is None:
            continue

        # Apply cosine similarity for textual comparison
        similarity = cosine_similarity(str(target_value), str(other_value))
        similarities.append(weight * similarity)
        existing_weights += weight  # Only sum up existing weights

    # If no valid components were compared, return 0
    if existing_weights == 0:
        return 0

    # Normalize by the sum of existing weights
    return sum(similarities) / existing_weights


def calculate_structural_similarity(target_node_data, other_node_data):
    """
    Menghitung kemiripan struktural antara dua node menggunakan Graph Edit Distance (GED).
    - Cost Insert, Delete, Substitusi = 1
    - Jika tipe node berbeda → Cost 2 (karena harus delete + insert)
    - Jika target node missing → Cost 1 (karena hanya perlu insert)
    - Jika tipe sama tapi kontennya beda → Cost 1 (karena hanya perlu subtitusi)
    
    Hasil Similarity:
    - Cost 0 → Similarity = 1.0 (100%)
    - Cost 1 → Similarity = 0.5 (50%)
    - Cost 2 → Similarity = 0.0 (0%)
    """
    
    cost_insert = 1
    cost_delete = 1
    cost_substitute = 1
    
    # Ambil tipe dan konten dari kedua node
    target_type = target_node_data.get('type', 'Unknown')
    other_type = other_node_data.get('type', 'Unknown')
    
    target_label = target_node_data.get('label', 'Unknown')
    other_label = other_node_data.get('label', 'Unknown')

    # Jika salah satu node hilang
    if not target_node_data or not other_node_data:
        cost = cost_insert
    # Jika tipe berbeda, maka butuh delete + insert
    elif target_type != other_type:
        cost = cost_delete + cost_insert  # Total cost = 2
    # Jika tipe sama tetapi label berbeda, hanya perlu subtitusi
    elif target_label != other_label:
        cost = cost_substitute
    # Jika sama persis, tidak ada cost
    else:
        cost = 0

    # Konversi cost ke similarity
    if cost == 0:
        similarity = 1.0  # 100% similarity
    elif cost == 1:
        similarity = 0.5  # 50% similarity
    else:
        similarity = 0.0  # 0% similarity

    print(f"Cost: {cost}, Structural Similarity: {similarity}")
    
    return similarity, cost


def detect_missing_nodes_by_count(graph):
    """
    Detects missing nodes in the graph based on their count.
    A node is considered missing if it does not appear at least twice in the graph.
    """
    node_counts = {}
    
    # Count occurrences of each node in edges (both source and target)
    for source, target in graph.edges():
        node_counts[source] = node_counts.get(source, 0) + 1
        node_counts[target] = node_counts.get(target, 0) + 1

    # Identify nodes with less than 2 occurrences
    missing_nodes = [node for node, count in node_counts.items() if count < 2 and node not in ("StartEvent", "EndEvent")]
    # print("node_counts:", node_counts)
    # print("Missing Node:", missing_nodes)
    return missing_nodes


def calculate_combined_similarity_with_ged(target_graph, target_node, other_graph, other_node, component):
    """
    Menghitung kemiripan kombinasi (semantic + structural) antar node.
    - **Semantic similarity** dihitung dengan cosine similarity.
    - **Structural similarity** dihitung dengan Graph Edit Distance (GED).
    """

    # Ambil data node
    target_node_data = target_node[1]
    other_node_data = other_node[1]

    # Hitung kemiripan semantic
    semantic_similarity = calculate_semantic_similarity(target_node_data, other_node_data, component)

    # Hitung kemiripan structural menggunakan GED
    structural_similarity, cost = calculate_structural_similarity(target_node_data, other_node_data)
    
    print(f"Semantic Similarity: {semantic_similarity}, Structural Similarity: {structural_similarity}")

    # Kombinasikan dengan bobot yang sama (50% semantic, 50% structural)
    return semantic_similarity, structural_similarity, cost


def compare_bpmn_with_missing_node_detection(target_data, other_data_list):
    """
    Membandingkan target graph node-by-node dengan graph lain,
    menggunakan GED Greedy untuk structural similarity dan semantic similarity,
    serta mendeteksi missing nodes langsung dari target graph.
    """
    target_graph = convert_to_interstructural_with_order(target_data)
    target_nodes = list(target_graph.nodes(data=True))
    
    best_graph = None
    best_graph_name = None
    best_similarity = 0
    best_semantic = 0
    best_structural = 0
    best_total_cost = 0
    best_missing_nodes = []
    best_cost = 0

    # Detect missing nodes from the target graph
    target_missing_nodes = detect_missing_nodes_by_count(target_graph)
    print("Target Missing Nodes:", target_missing_nodes)

    for other_data in other_data_list:
        current_missing_nodes = []
        print(f"\nComparing with: {other_data['file_name']}")
        other_graph = convert_to_interstructural_with_order(other_data)
        other_nodes = list(other_graph.nodes(data=True))

        print(f"{other_data['file_name']} Nodes:", list(other_graph.nodes()))

        total_semantic_similarity_graph = 0
        total_structural_similarity_graph = 0
        total_cost_graph = 0
        total_nodes_compared_graph = 0
        total_similarity = 0
        valid = True

        i, j = 0, 0
        while i < len(target_nodes) and j < len(other_nodes):
            target_nc = target_nodes[i]
            other_nc = other_nodes[j]

            component_type = 'activities' if target_nc[1].get('type') != 'Transition' else 'transitions'

            if target_nc[0] in target_missing_nodes:
                print(f"Missing node detected: {target_nc[0]} - Reason: Consecutive non-Transition nodes")

                match_found = False
                for k in range(j, len(other_nodes)):
                    other_nc_next = other_nodes[k]
                    semantic_similarity, structural_similarity, cost = calculate_combined_similarity_with_ged(
                        target_graph, target_nc, other_graph, other_nc_next, component_type
                    )
                    
                    total_cost_graph += cost

                    combined_similarity_next = 0.5 * semantic_similarity + 0.5 * structural_similarity

                    print(f"{target_nc[0]} vs {other_nc_next[0]}: {combined_similarity_next}")

                    if combined_similarity_next > 0.5:
                        print(f"Match {target_nc[0]} vs {other_nc_next[0]}: {combined_similarity_next}")
                        match_found = True
                        total_similarity += combined_similarity_next
                        total_nodes_compared_graph += 1

                        total_semantic_similarity_graph += semantic_similarity
                        total_structural_similarity_graph += structural_similarity

                        j = k  
                        break
                    else:
                        current_missing_nodes.append(other_nc_next[0])

                if not match_found:
                    print(f"Disqualifying graph: {other_data['file_name']} - Reason: Could not find match for missing node {target_nc[0]}")
                    valid = False
                    break
            
            else:
                semantic_similarity, structural_similarity, cost = calculate_combined_similarity_with_ged(
                    target_graph, target_nc, other_graph, other_nc, component_type
                )
                
                total_cost_graph += cost

                combined_similarity = 0.5 * semantic_similarity + 0.5 * structural_similarity

                print(f"Current: {target_nc[0]} vs {other_nc[0]} - Similarity: {combined_similarity}")

                if combined_similarity < 0.5:
                    print(f"Disqualifying graph: {other_data['file_name']} at node: {target_nc[0]} vs {other_nc[0]} - Similarity below threshold")
                    valid = False
                    break

                total_similarity += combined_similarity
                total_nodes_compared_graph += 1

                total_semantic_similarity_graph += semantic_similarity
                total_structural_similarity_graph += structural_similarity

            i += 1
            j += 1

        if valid and total_nodes_compared_graph > 0:
            avg_similarity = total_similarity / total_nodes_compared_graph
            avg_semantic_similarity_graph = (total_semantic_similarity_graph / total_nodes_compared_graph)
            avg_structural_similarity_graph = (total_structural_similarity_graph / total_nodes_compared_graph)

            print(f"\nGraph: {other_data['file_name']} - Average Combined Similarity: {avg_similarity:.2f}")
            print(f"Graph: {other_data['file_name']} - Total Semantic Similarity: {avg_semantic_similarity_graph}")
            print(f"Graph: {other_data['file_name']} - Total Structural Similarity: {avg_structural_similarity_graph}")
            print(f"Graph: {other_data['file_name']} - Total Cost: {total_cost_graph}")

            if avg_similarity > best_similarity:
                best_graph = other_graph
                best_graph_name = other_data.get('file_name', 'Unknown')
                best_similarity = avg_similarity
                best_semantic = avg_semantic_similarity_graph
                best_structural = avg_structural_similarity_graph
                best_missing_nodes = current_missing_nodes
                best_cost = total_cost_graph
                
                if j < len(other_nodes):
                    best_missing_nodes.extend(k[0] for k in other_nodes[j:])
    
    print("OEOE ", target_data)
    print(f"\nFinal Results: {target_data.get('file_name', 'Unknown')}")
    print("Best Graph:", best_graph_name)
    print("Best Similarity:", best_similarity)
    print("Best Semantic:", best_semantic)
    print("Best Structural:", best_structural)
    print("Best Cost:", best_cost)
    print(f"Target Missing Nodes: {target_missing_nodes}")
    print("Matching nodes:", best_missing_nodes)
    
    return best_graph, best_graph_name, target_graph



# Visualize graph with updated labels
def visualize_graph_with_labels(G, title="Graph Visualization", file_name="Graph.png"):
    # Update node labels to include "Start" and "End"
    # label_start_and_end_nodes(G)

    # Use the updated labels for visualization
    pos = nx.spring_layout(G)
    labels = nx.get_node_attributes(G, 'label')
    # Draw the graph with labels
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, labels=labels, 
            node_color='lightblue', edge_color='black', node_size=1000, font_size=12, font_weight='bold')
    
    # Set title and adjust layout
    plt.title(title, fontsize=14)
    # plt.tight_layout()
    plt.savefig(file_name)
    plt.show()


if __name__ == "__main__":
    folder_path = 'Dataset repositori'
    target_file = 'XPDL Missing/BPM 1 EquipmentRecap.xpdl'

    # Parse target file
    target_data = parse_xpdl(target_file)
    target_data['file_name'] = target_file

    # Parse other files
    other_data_list = []
    for file_name in os.listdir(folder_path):
        if (file_name.endswith(".xpdl") or file_name.endswith(".xml"))  and file_name != os.path.basename(target_file):
            file_path = os.path.join(folder_path, file_name)
            parsed_data = parse_xpdl(file_path)
            parsed_data['file_name'] = file_name  # Add file name
            other_data_list.append(parsed_data)
    
    # Compare BPMNs
    best_graph, best_graph_name, target_graph = compare_bpmn_with_missing_node_detection(target_data, other_data_list)

    # Visualize the main graph
    visualize_graph_with_labels(target_graph, f"Graph Visualization of Target XPDL: {target_file}", f"Visual/Target.png")
    if best_graph:
        visualize_graph_with_labels(best_graph, f"Graph Visualization of Best Match: {best_graph_name}", f"Visual/Match.png")


⚠️ Missing transitions detected! Attempting to reconstruct...
  - input data to system (effd3951-491a-4de1-8cb7-986fb61d7ff7) is missing connections.
  ✅ Reconstructed missing transition: input data to system → send to the store
  - send to the store (165df246-4ec5-4af4-b8f5-5726c93fe9aa) is missing connections.
  ✅ Reconstructed missing transition: send to the store → EndEvent
  - EndEvent (61a9001b-9a33-4790-b582-1af76f631d94) is missing connections.
Final Transitions: XPDL Missing/BPM 1 EquipmentRecap.xpdl [{'from': 'StartEvent', 'to': 'put a stamp on each tool'}, {'from': 'input data to system', 'to': 'send to the store'}, {'from': 'send to the store', 'to': 'EndEvent'}]
Final Transitions: Dataset repositori\Add new item.xpdl [{'from': 'StartEvent', 'to': 'Asks to add new item'}, {'from': 'Asks to add new item', 'to': 'Displays form of  add new item'}, {'from': 'Displays form of  add new item', 'to': 'Enters a barcode'}, {'from': 'Enters a barcode', 'to': 'Queries for goods in the

Final Transitions: Dataset repositori\Change work unit.xpdl [{'from': 'StartEvent', 'to': 'Asks to move the member work unit'}, {'from': 'Asks to move the member work unit', 'to': 'Displays a form to move work units'}, {'from': 'Displays a form to move work units', 'to': '95886389-3a7a-49ce-97a0-9083d139f641'}, {'from': '95886389-3a7a-49ce-97a0-9083d139f641', 'to': 'Fills in information about the member '}, {'from': 'Fills in information about the member ', 'to': 'searches information of member '}, {'from': 'searches information of member ', 'to': 'c8cb5129-5c20-4f71-a3f3-a87f8b827264'}, {'from': 'c8cb5129-5c20-4f71-a3f3-a87f8b827264', 'to': 'Displays information  of member'}, {'from': 'c8cb5129-5c20-4f71-a3f3-a87f8b827264', 'to': '95886389-3a7a-49ce-97a0-9083d139f641'}, {'from': 'Displays information  of member', 'to': 'Enters a new work unit'}, {'from': 'Enters a new work unit', 'to': 'Verifies information of new work '}, {'from': 'Verifies information of new work ', 'to': 'afc65e35-

Final Transitions: Dataset repositori\Display cash inflow.xpdl [{'from': 'StartEvent', 'to': 'Wants to display cash inflow'}, {'from': 'Wants to display cash inflow', 'to': 'Displays the cash inflow form'}, {'from': 'Displays the cash inflow form', 'to': 'Fills in the year and month '}, {'from': 'Fills in the year and month ', 'to': 'Queries about the cash inflow in the cash database'}, {'from': 'Queries about the cash inflow in the cash database', 'to': 'Displays the query result '}, {'from': 'Displays the query result ', 'to': 'bf8db104-94fe-43d9-b858-43e2e47663dd'}, {'from': 'bf8db104-94fe-43d9-b858-43e2e47663dd', 'to': 'Prints a report to the printer'}, {'from': 'bf8db104-94fe-43d9-b858-43e2e47663dd', 'to': 'e8510fe5-4429-4b46-8ca7-bd0ad5201531'}, {'from': 'Prints a report to the printer', 'to': 'e8510fe5-4429-4b46-8ca7-bd0ad5201531'}, {'from': 'e8510fe5-4429-4b46-8ca7-bd0ad5201531', 'to': 'EndEvent'}, {'from': 'e7fc5953-e788-4e5b-80b8-d1629e69b68f', 'to': 'data'}]
Final Transition

Final Transitions: Dataset repositori\EncodeTag.xpdl [{'from': 'StartEvent', 'to': 'open the RFID reader application'}, {'from': 'open the RFID reader application', 'to': 'input the parent number id into the rfid tag'}, {'from': 'input the parent number id into the rfid tag', 'to': 'save the master number into the RFID tag'}, {'from': 'save the master number into the RFID tag', 'to': 'EndEvent'}]
Final Transitions: Dataset repositori\exhibition_organized_by_the_library.xpdl [{'from': 'StartEvent', 'to': 'identifies the needs.'}, {'from': 'identifies the needs.', 'to': 'classified the needs'}, {'from': 'classified the needs', 'to': 'lists the prioritize the needs'}, {'from': 'lists the prioritize the needs', 'to': 'verified the specifications of the needs'}, {'from': 'verified the specifications of the needs', 'to': '77b8d9f6-964f-4087-be27-e25ea61a13db'}, {'from': '77b8d9f6-964f-4087-be27-e25ea61a13db', 'to': 'selects material'}, {'from': '77b8d9f6-964f-4087-be27-e25ea61a13db', 'to': '


⚠️ Missing transitions detected! Attempting to reconstruct...
  - EndEvent (57fd3017-c97a-4038-94cd-26805f10a24a) is missing connections.
  ✅ Reconstructed missing transition: final result → EndEvent
  - final result (8cf90868-ee5a-483e-90be-ad0810080816) is missing connections.
Final Transitions: Dataset repositori\GeneralCollection.xpdl [{'from': 'StartEvent', 'to': 'Come directly to the General Collection\xa0'}, {'from': 'Come directly to the General Collection\xa0', 'to': 'Scan ID card barcode'}, {'from': 'Scan ID card barcode', 'to': 'Search in opac'}, {'from': 'Search in opac', 'to': 'Found/not?'}, {'from': 'Found/not?', 'to': 'Reading / borrowing'}, {'from': 'Found/not?', 'to': 'Ask the staff for help'}, {'from': 'Reading / borrowing', 'to': 'read/borrow?'}, {'from': 'Ask the staff for help', 'to': 'Do a further search'}, {'from': 'read/borrow?', 'to': 'Reading on the spot'}, {'from': 'read/borrow?', 'to': 'Make a loan'}, {'from': 'Do a further search', 'to': '28938ac7-a5c0-4a0

...

Final Transitions: Dataset repositori\Review Application Processing.xpdl [{'from': 'StartEvent', 'to': 'Check the Application Form'}, {'from': 'Check the Application Form', 'to': '17facf36-0e57-4c62-9ede-e99923624ed4'}, {'from': '17facf36-0e57-4c62-9ede-e99923624ed4', 'to': 'Validation ID Card And Family Card'}, {'from': '17facf36-0e57-4c62-9ede-e99923624ed4', 'to': 'Validation he employment information form,the bank statement and\xa0 the previous loan statement'}, {'from': 'Validation ID Card And Family Card', 'to': 'e10a4d07-f9b6-408f-90b1-d2e78d65b9b0'}, {'from': 'Validation he employment information form,the bank statement and\xa0 the previous loan statement', 'to': 'e10a4d07-f9b6-408f-90b1-d2e78d65b9b0'}, {'from': 'e10a4d07-f9b6-408f-90b1-d2e78d65b9b0', 'to': 'Approve the Loan Approcation'}, {'from': 'Approve the Loan Approcation', 'to': 'Reduce the Amount'}, {'from': 'Reduce the Amount', 'to': 'Change the interest rate'}, {'from': 'Change the interest rate', 'to': 'Receive the Fu

Final Transitions: Dataset repositori\StokOpnameService.xpdl [{'from': 'StartEvent', 'to': 'Composing ROP'}, {'from': 'Composing ROP', 'to': 'carry out stock opname preparation meeting'}, {'from': 'carry out stock opname preparation meeting', 'to': 'carry out stock opname'}, {'from': 'carry out stock opname', 'to': 'Carry out monitoring and evaluation'}, {'from': 'Carry out monitoring and evaluation', 'to': 'Make a stock opname report'}, {'from': 'Make a stock opname report', 'to': 'make news event stock opname'}, {'from': 'make news event stock opname', 'to': 'EndEvent'}]
Final Transitions: Dataset repositori\submitting bibliography into ailis.xpdl [{'from': 'StartEvent', 'to': 'enter master number'}, {'from': 'enter master number', 'to': 'determine the classification number'}, {'from': 'determine the classification number', 'to': 'determine the subject title'}, {'from': 'determine the subject title', 'to': 'specify the title of the main entry'}, {'from': 'specify the title of the mai

Final Transitions: Dataset repositori\View items sold.xpdl [{'from': 'StartEvent', 'to': 'Asks to view items sold'}, {'from': 'Asks to view items sold', 'to': 'Queries goods on Category database'}, {'from': 'Queries goods on Category database', 'to': 'Displays form of view items sold'}, {'from': 'Displays form of view items sold', 'to': 'Chooses the year, month, and category'}, {'from': 'Chooses the year, month, and category', 'to': '4ed1996d-ecb5-4004-8591-becdb0e726e1'}, {'from': '4ed1996d-ecb5-4004-8591-becdb0e726e1', 'to': 'Queries the Stock database'}, {'from': '4ed1996d-ecb5-4004-8591-becdb0e726e1', 'to': '2f754b08-9a95-43cd-9848-fa7e279ad013'}, {'from': 'Queries the Stock database', 'to': 'Displays the query results to the table'}, {'from': '2f754b08-9a95-43cd-9848-fa7e279ad013', 'to': '0e9a5fc3-5fd7-47b9-965a-d02b258051e8'}, {'from': 'Displays the query results to the table', 'to': '2f754b08-9a95-43cd-9848-fa7e279ad013'}, {'from': '0e9a5fc3-5fd7-47b9-965a-d02b258051e8', 'to': '

Cost: 2, Structural Similarity: 0.0
Semantic Similarity: 0.0, Structural Similarity: 0.0
put a stamp on each tool vs Transition_Receive the payment certificate_to_EndEvent: 0.0
Cost: 1, Structural Similarity: 0.5
Semantic Similarity: 0.5, Structural Similarity: 0.5
put a stamp on each tool vs EndEvent: 0.5
Disqualifying graph: Closing Process.xpdl - Reason: Could not find match for missing node put a stamp on each tool

Comparing with: collection_services_of_scientific_papers.xpdl
parsed_data['transitions'] [{'from': 'StartEvent', 'to': 'member submits the scientific paper'}, {'from': 'member submits the scientific paper', 'to': 'checks the completeness of scientific paper'}, {'from': 'checks the completeness of scientific paper', 'to': 'paper complete'}, {'from': 'paper complete', 'to': 'receives the scientific paper'}, {'from': 'paper complete', 'to': 'reject the scientific paper'}, {'from': 'receives the scientific paper', 'to': 'fills out the scientific paper submission form'}, {'f

Cost: 2, Structural Similarity: 0.0
Semantic Similarity: 0.0668153104781061, Structural Similarity: 0.0
input data to system vs Transition_Queries the current month's sales _to_Displays the query results to a table: 0.03340765523905305
Cost: 2, Structural Similarity: 0.0
Semantic Similarity: 0.0625, Structural Similarity: 0.0
input data to system vs Transition_Queries the daily recap of this month _to_Displays the query results to a table: 0.03125
Cost: 2, Structural Similarity: 0.0
Semantic Similarity: 0.0, Structural Similarity: 0.0
input data to system vs Transition_Enters the spesific date _to_Queries the spesific date: 0.0
Cost: 1, Structural Similarity: 0.5
Semantic Similarity: 0.5, Structural Similarity: 0.5
input data to system vs Queries the spesific date: 0.5
Cost: 2, Structural Similarity: 0.0
Semantic Similarity: 0.0944911182523068, Structural Similarity: 0.0
input data to system vs Transition_Displays the query results to a table_to_None_593d01ff-9bdd-4d97-8708-3e5155ce4f3